In [11]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from pipeline import create_pipeline
import os
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [12]:
import time

In [13]:
# labeling
# Get Dataset
files={
    'CICI':'/home/irteam/junghye-dcloud-dir/MLAC/data/encoded_ConcatedCICI.csv',
    'UNSW': '/home/irteam/junghye-dcloud-dir/MLAC/data/encoded_ConcatedUNSW.csv'
}

data = pd.read_csv(files['UNSW'])
data=data[np.isfinite(data).all(1)]
multiclass_labels=data['new_category']

data=data.drop(labels=['label','attack_category','nist_category','new_category','Unnamed: 0'],axis=1)

In [14]:
# Define Models
models = []
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('ABoost', AdaBoostClassifier()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('MLP', MLPClassifier()))

In [15]:
df=pd.DataFrame(columns=['name','acc','f1_mi','f1_ma','f1_we','recall_mi','recall_ma','recall_we']+\
                 ['precision_mi','precision_ma','precision_we'])
eval_path='/home/irteam/junghye-dcloud-dir/MLAC/evaluation'
confusion_path='/home/irteam/junghye-dcloud-dir/MLAC/confusion_matrix/new_unsw'
if os.path.isdir(confusion_path)==False:
    os.mkdir(confusion_path)
cnt=0

In [16]:
X_train,X_test,y_train,y_test=train_test_split(data,multiclass_labels,test_size=0.3, shuffle=True, stratify=multiclass_labels, random_state=34)


In [17]:
import matplotlib.pyplot as plt

In [18]:
# confusion matrix plot
def plot_confusion_matrix(con_mat,labels,title:str,cmap=plt.cm.get_cmap('Blues'),normalize=False):
    plt.imshow(con_mat,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    marks=np.arange(len(labels))
    nlabels=[]
    for k in range(len(con_mat)):
        n=sum(con_mat[k])
        nlabel='{0}(n={1})'.format(labels[k],n)
        nlabels.append(nlabel)

    plt.xticks(marks,labels,rotation=45)
    plt.yticks(marks,nlabels)

    thresh=con_mat.max()/2.
    if normalize:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, '{0}%'.format(con_mat[i, j] * 100 / n), horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    else:
        for i, j in itertools.product(range(con_mat.shape[0]), range(con_mat.shape[1])):
            plt.text(j, i, con_mat[i, j], horizontalalignment="center", color="white" if con_mat[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    #plt.show()
    #이미지 저장
    plt.savefig(confusion_path+'/'+title+'.png',facecolor='#eeeeee',edgecolor='blue',pad_inches=0.5)
    plt.clf()

In [ ]:
for name, model in models:

    model=create_pipeline(model)
    print('training start...'+name)
    start_time=time.time()
    model.fit(X_train,y_train)
    end_time=time.time()
    
    print(name+" training 소요 시간: {:.2f}초".format(end_time - start_time))


    #evaluation
    
    print('evaluation start...')
    start_time=time.time()
    y_pred=model.predict(X_test)
    end_time=time.time()
    print(name+" test 소요 시간: {:.2f}초".format(end_time - start_time))

    #evaluation result
    model_eval=[]
    model_eval.append(name)
    
    acc = accuracy_score(y_test, y_pred)
    f1_mi = f1_score(y_test, y_pred,average='micro')
    f1_ma = f1_score(y_test, y_pred,average='macro')
    f1_we = f1_score(y_test, y_pred,average='weighted')
    recall_mi = recall_score(y_test, y_pred, average='micro')
    recall_ma = recall_score(y_test, y_pred, average='macro')
    recall_we = recall_score(y_test, y_pred, average='weighted')
    precision_mi = precision_score(y_test, y_pred, average='micro')
    precision_ma = precision_score(y_test, y_pred, average='macro')
    precision_we = precision_score(y_test, y_pred, average='weighted')
    
    model_eval.append(acc)
    model_eval.append(f1_mi)
    model_eval.append(f1_ma)
    model_eval.append(f1_we)
    model_eval.append(recall_mi)
    model_eval.append(recall_ma)
    model_eval.append(recall_we)
    model_eval.append(precision_mi)
    model_eval.append(precision_ma)
    model_eval.append(precision_we)
    
    #confusion_metrics
    confusion=metrics.confusion_matrix(y_test,y_pred)
    plot_confusion_matrix(confusion,labels=['Benign','DoS','Reconnaissance']+\
                          ['Malware','Unauthorized access','Generic','Fuzzers'],title=name)
       


    print(f'name:{name},acc:{acc},f1_score:{f1_mi},{f1_ma},{f1_we},recall:{recall_mi},{recall_ma},{recall_we},precision:{precision_mi},{precision_ma},{precision_we}')
    df.loc[cnt]=model_eval

    cnt+=1
    

df.to_csv(os.path.join(eval_path,'new_unsw.csv'),index=False)

training start...RF
RF training 소요 시간: 17.33초
evaluation start...
RF test 소요 시간: 3.50초
name:RF,acc:0.8744289074734458,f1_score:0.8744289074734458,0.14288594142862587,0.8164260350820199,recall:0.8744289074734458,0.1478289528140559,0.8744289074734458,precision:0.8744289074734458,0.26563808855911825,0.7813222007306004
training start...CART
CART training 소요 시간: 43.15초
evaluation start...
CART test 소요 시간: 3.17초
name:CART,acc:0.9686731919306258,f1_score:0.9686731919306258,0.37077743510331057,0.9590547613834819,recall:0.9686731919306258,0.3877869561173693,0.9686731919306258,precision:0.9686731919306258,0.6968599711245977,0.9636611988507532
training start...NB
NB training 소요 시간: 17.59초
evaluation start...
NB test 소요 시간: 9.53초
name:NB,acc:0.8760542866076412,f1_score:0.8760542866076412,0.19000355402668562,0.8247956260408833,recall:0.8760542866076412,0.19292379793705525,0.8760542866076412,precision:0.8760542866076412,0.37913626955861096,0.7962253744158936
training start...LDA
LDA training 소요 시간: 